In [11]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
def build_data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):

    transform = transforms.Compose([transforms.ToTensor()])
    
    train_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    valid_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)
  
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_loader, valid_loader, test_loader)

In [3]:
def train(model, data_loader, valid_loader, num_epochs, criterion, optimizer, device):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for step, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
               
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            

In [4]:
def validate(model, valid_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
    print('Accuracy of the network on the {} validation images: {:.2f} %'.format(5000, 100 * correct / total)) 

In [5]:
def test(model, test_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))  

In [6]:
def count_parameters(model):
    parameters = list(model.parameters())
    total_parms = sum([np.prod(p.size()) for p in parameters if p.requires_grad])
    return total_parms


In [7]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc1 = nn.Sequential(
            nn.Linear(64*7*7, 512),
            nn.Dropout(0.25))
        
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [9]:
# General parameters
data_dir = '/tmp'
device = 'cpu'
num_classes = 10

# Hyperparameters
max_lr = 0.00001
weight_decay = 0.005
batch_size = 64
learning_rate = 0.0001
num_epochs = 1
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# FashionMNIST dataset 
train_loader, valid_loader, test_loader = build_data_loader(data_dir=data_dir, batch_size=batch_size)

# Model definition
model = CNN()

# Optimizer
optimizer = optimizer(model.parameters(), max_lr, weight_decay=weight_decay)

In [ ]:
print(count_parameters(model))

138357544


In [ ]:
%%time
# Train the model
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
    with record_function("training"):
        train(model, train_loader, valid_loader, num_epochs, criterion, optimizer, device)
        
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

Step [1/844], Loss: 2.4562
Step [2/844], Loss: 2.3195
Step [3/844], Loss: 2.3232
Step [4/844], Loss: 2.3027
Step [5/844], Loss: 2.2284
Step [6/844], Loss: 2.3011
Step [7/844], Loss: 2.2842
Step [8/844], Loss: 2.2664
Step [9/844], Loss: 2.2091
Step [10/844], Loss: 2.1720
Step [11/844], Loss: 2.1101
Step [12/844], Loss: 2.1202
Step [13/844], Loss: 2.0983
Step [14/844], Loss: 2.0700
Step [15/844], Loss: 2.0615
Step [16/844], Loss: 1.9833
Step [17/844], Loss: 2.0228
Step [18/844], Loss: 1.9637
Step [19/844], Loss: 1.9869
Step [20/844], Loss: 1.9376
Step [21/844], Loss: 1.9018
Step [22/844], Loss: 1.9081
Step [23/844], Loss: 1.8654
Step [24/844], Loss: 1.8457
Step [25/844], Loss: 1.8280
Step [26/844], Loss: 1.8223
Step [27/844], Loss: 1.7437
Step [28/844], Loss: 1.7903
Step [29/844], Loss: 1.7964
Step [30/844], Loss: 1.7667
Step [31/844], Loss: 1.8466
Step [32/844], Loss: 1.7255
Step [33/844], Loss: 1.6633
Step [34/844], Loss: 1.7211
Step [35/844], Loss: 1.6883
Step [36/844], Loss: 1.6112
S

In [ ]:
test(model, test_loader, device)